In [1]:
# !pip uninstall transformers -y
# !pip install transformers
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
# !pip install accelerate -U
# !pip install evaluate
# !pip install tensorrt
# !pip install tensorflow
# !pip install ipywidgets


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"

In [ ]:
# !pip uninstall transformers -y
# !pip install git+https://github.com/pavi-ninjaac/transformers.git@pavithra_cant_convert_to_egertensor3#egg=transformers
    
# import transformers

# transformers.__version__ # 4.5.1

In [2]:
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict, Dataset
import datasets

from transformers.tokenization_utils import BatchEncoding


multi_news_datasets: DatasetDict = load_dataset("alexfabbri/multi_news")
# d: dict = multi_news_dataset["train"][:1000] # can't use map function for this so
# data = Dataset.from_dict({"document": multi_news_dataset["train"]["document"][:1000], "summary": multi_news_dataset["train"]["summary"][:1000]})
# print(type(data))
multi_news_datasets_small: DatasetDict = datasets.DatasetDict({"train": Dataset.from_dict({"document": multi_news_datasets["train"]["document"][:100],
                                                                                            "summary": multi_news_datasets["train"]["summary"][:100]}),
                                                               "validation": Dataset.from_dict({"document": multi_news_datasets["validation"]["document"][:100],
                                                                                            "summary": multi_news_datasets["validation"]["summary"][:100]}),
                                                                "test": Dataset.from_dict({"document": multi_news_datasets["test"]["document"][:100],
                                                                                            "summary": multi_news_datasets["test"]["summary"][:100]})})


/home/pavithra/projects/Natural-Language-Processing/venv_text_summarizaition/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for alexfabbri/multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/alexfabbri/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
# gonna use this model
# Load model directly
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# model_checkpoint = "google/mt5-small"
model_checkpoint = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, )
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,from_pt=True)


2024-05-29 21:09:39.798871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 21:09:39.836480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 21:09:40.738880: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [6]:
model.summary()

Model: "tf_pegasus_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  570797056 
                                                                 
 final_logits_bias (BiasLay  multiple                  96103     
 er)                                                             
                                                                 
Total params: 570893159 (2.13 GB)
Trainable params: 570797056 (2.13 GB)
Non-trainable params: 96103 (375.40 KB)
_________________________________________________________________


In [7]:
# play with the tockenizer a little bit
a = tokenizer("somehting for my understanding")
tokenizer.convert_ids_to_tokens(a["input_ids"])

['▁some', 'h', 'ting', '▁for', '▁my', '▁understanding', '</s>']

In [8]:
a

{'input_ids': [181, 1467, 4011, 118, 161, 1301, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenizer.model_max_length

1024

In [4]:
# create a collate function and do the tokinizarion.
max_input_length: int = 900
max_target_length: int = 124

def tokenize_sentence(dataset: DatasetDict) -> BatchEncoding:
    # datasets: LazyBatch datatype
    model_input: BatchEncoding = tokenizer(dataset["document"],
                                           max_length=max_input_length,
                                           truncation=True, 
                                           return_tensors="tf",
                                           padding=True)
    labels = tokenizer(dataset["summary"],
                       max_length=max_target_length,
                       truncation=True,
                       return_tensors="tf",
                       padding=True)
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_data: DatasetDict = multi_news_datasets_small.map(tokenize_sentence, batched=True)
print(type(tokenized_data))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<class 'datasets.dataset_dict.DatasetDict'>


In [11]:
# save the tokenizer
tokenizer.save_pretrained("tokenizer")


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

# using keras

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# padding.
from transformers import DataCollatorForSeq2Seq
# Next, we need to define a data collator for our sequence-to-sequence task. Since mT5 is an encoder-decoder Transformer model,
# one subtlety with preparing our batches is that during decoding we need to shift the labels to the right by one. This is required
# to ensure that the decoder only sees the previous ground truth labels and not the current or future ones, which would be easy for
# the model to memorize. This is similar to how masked self-attention is applied to the inputs in a task like causal language modeling.
# from transformers import DataCollatorForSeq2Seq

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

print(tokenized_data)



def data_collator_function(tokenizer, features, padding,  max_length, label_name: str):

    # pad the non-label columns.
    print(features)
    non_labels_features = [{k: v for k, v in feature.items() if k != label_name} for feature in features]
    print(non_labels_features)
    labels_features = [features[label_name]]
    # To avoid errors when using Feature extractors
    if not hasattr(tokenizer, "deprecation_warnings"):
        return tokenizer.pad(non_labels_features, padding=padding, max_length=max_length, return_tensors="tf")

    # Save the state of the warning, then disable it
    warning_state = tokenizer.deprecation_warnings.get("Asking-to-pad-a-fast-tokenizer", False)
    tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

    try:
        padded = tokenizer.pad(non_labels_features, padding=padding, max_length=max_length)
    finally:
        # Restore the state of the warning.
        tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = warning_state

    # pad the 
    return padded


features = [tokenized_data["train"][i] for i in range(2)]
# data_collator(features, return_tensors="np")

#data_collator_function(tokenizer, features=features, max_length=max_target_length, padding=True, label_name="labels")


DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [14]:
tokenizer.pad_token_id

0

In [6]:
# need to remove the column names, the collate funciton doesn't know how to pad the string columes -- remove the string columns
tokenized_data = tokenized_data.remove_columns(
    multi_news_datasets["train"].column_names
)

tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [16]:
[tokenized_data["train"][i] for i in range(2)]

[{'input_ids': [765,
   11980,
   2657,
   108,
   126,
   123,
   116,
   120,
   166,
   435,
   108,
   3428,
   107,
   168,
   123,
   116,
   109,
   211,
   1197,
   113,
   109,
   625,
   108,
   173,
   118,
   156,
   521,
   121,
   2770,
   121,
   56904,
   1171,
   109,
   2656,
   113,
   2948,
   1411,
   108,
   1741,
   111,
   4449,
   1411,
   127,
   149,
   10237,
   124,
   156,
   474,
   151,
   8108,
   107,
   202,
   1163,
   1597,
   124,
   109,
   475,
   107,
   283,
   107,
   2856,
   1288,
   118,
   1268,
   4341,
   109,
   9189,
   134,
   14087,
   114,
   107,
   208,
   107,
   351,
   859,
   166,
   1192,
   156,
   113,
   109,
   205,
   356,
   34375,
   124,
   199,
   109,
   1968,
   53597,
   333,
   109,
   1331,
   625,
   107,
   48208,
   127,
   118,
   599,
   17233,
   177,
   1665,
   112,
   129,
   732,
   108,
   992,
   112,
   20061,
   42259,
   141,
   16393,
   3852,
   77274,
   116,
   108,
   1711,
   112,
   2168,
 

In [7]:
#for example
# features = [tokenized_data["train"][i] for i in range(1)]
import numpy as np
fetures = [{'input_ids': np.array([259, 23129, 259, 91074, 1]),
  'attention_mask': np.array([1, 1, 1, 1, 1]),
  'labels': np.array([322, 1])},
  {'input_ids': np.array([259, 23129, 259, 91074, 1]),
  'attention_mask': np.array([1, 1, 1, 1, 1]),
  'labels': np.array([222, 1])}]
#print(fetures)
data_collator(fetures, return_tensors="tf")


[]
0
2
[322   1]
[]
0
2
[222   1]


{'input_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[  259, 23129,   259, 91074,     1],
       [  259, 23129,   259, 91074,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]], dtype=int32)>, 'labels': <tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[322,   1],
       [222,   1]])>, 'decoder_input_ids': <tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[  0, 322],
       [  0, 222]])>}

In [16]:
np.concatenate([[], np.array([233,1])])

array([233.,   1.])

In [18]:

label = np.array([233, 1])
max_label_length = 4
label_pad_token_id = -100
np.concatenate([[label_pad_token_id] * (max_label_length - len(label)), label])


array([-100, -100,  233,    1])

In [19]:
label = np.array([233, 1])
max_label_length = 2
label_pad_token_id = -100
np.concatenate([[label_pad_token_id] * (max_label_length - len(label)), label])


array([233.,   1.])

In [24]:

label = np.array([233, 1])
max_label_length = 2
label_pad_token_id = -100
np.concatenate([np.array([label_pad_token_id] * (max_label_length - len(label)), dtype=np.int64),label])


array([233,   1])

In [18]:

tf_train_dataset = model.prepare_tf_dataset(
    tokenized_data["train"],
    collate_fn=data_collator,
    tokenizer=tokenizer,
    shuffle=True,
    batch_size=8,
)



In [19]:
tf_train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, 900), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, 900), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(8, 124), dtype=tf.int64, name=None)}, TensorSpec(shape=(8, 124), dtype=tf.int64, name=None))>

In [20]:
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_data["validation"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

In [21]:
tf_eval_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 900), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 900), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(None, 124), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 124), dtype=tf.int64, name=None))>

In [22]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")





The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [23]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-multi-news", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=8
)


/home/pavithra/projects/Natural-Language-Processing/venv_text_summarizaition/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/home/pavithra/projects/Natural-Language-Processing/project_3_text_summarization/check_the_work/pegasus-cnn_dailymail-finetuned-multi-news is already a clone of https://huggingface.co/pavithra-devi/pegasus-cnn_dailymail-finetuned-multi-news. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/8


: 

In [12]:
# import tensorflow as tf
import numpy as np
# a = [np.array([1,2]), np.array([3,4]), np.array([5,6])] # works good when the values as in the same type
# a = [np.array([1. ,2.])] # failing when converting from float to integer

# # give the values not as a list.
# aa = np.array([0,0])
# for array in a:
#     aa = np.vstack([aa, array])
#     print(aa)
# aaa = (np.vstack([aa, array]) for array in a)
# print(aaa)
# print(a)
# print(aa)
# # aa = [[2., 1.]]
# tf.constant([[1,2]], dtype=tf.int64)


# goal: convert this to a tensor.
# a = [np.array([322.,   1.]), np.array([322.,   2.])]
# a_single = [np.array([322.,   1.])]


# print(np.stack(a))
# print(np.stack(a_single))
# numpy values:  [array([322.,   1.]), array([322.,   1.]), array([322.,   1.]), array([322.,   1.]), array([322.,   1.]), array([322.,   1.]), array([322.,   1.]), array([322.,   1.])]


# 2 | Using the Trainer API (pytorch)

In [14]:
# train the model.
from transformers import AutoModelForSeq2SeqLM
model_checkpoint = "google/pegasus-cnn_dailymail"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)



Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# set the device tpe
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [16]:
# login to huggingface to push your model.

from huggingface_hub import notebook_login

notebook_login()
# huggingface-cli login --terminal cmd

In [17]:
# !huggingface-cli login

In [18]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_data["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    # predict_with_generate argument has been set to indicate that we should generate summaries
    # during evaluation so that we can compute ROUGE scores for each epoch
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

/home/pavithra/.local/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
import numpy as np
from nltk.tokenize import sent_tokenize
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    print(eval_pred)
    print(type(eval_pred))
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}


In [20]:
# data collater function.
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# need to remove the column names, the collate funciton doesn't know how to pad the string columes -- remove the string columns
# tokenized_data = tokenized_data.remove_columns(
#     multi_news_datasets["train"].column_names
# )
# tokenized_data

In [21]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [22]:
import tensorflow
tensorflow.__version__


'2.13.1'

In [23]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

: 

In [ ]:
https://discuss.huggingface.co/t/error-in-model-prepare-tf-dataset/88825